# Content-based Filtering

Aproach:
- Table with features erstellen, ratings in zweitem table

- sklearn-kNN mit minkowski-distance auf features anwenden, erhalte matrix mit allen Nachbarn eines jeden Films inklusive Abstand

- Wähle für jeden Nutzer und Film die x-nächsten Nachbarn aus, die der Nutzer bewertet hat

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

In [46]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
import time
kwargs = dict(random_state=42)

In [47]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

## Feature Preprocessing

In [48]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [49]:
ratings

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0
...,...,...,...,...
787536,812812,1032,tt0083530,3.0
787537,812813,99,tt0107798,3.0
787538,812814,333,tt0093857,3.0
787539,812815,49,tt0144168,3.0


In [50]:
omdb

,0,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,...,Series,Released_season,Released_month,Released_day,PG_Rating,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated,Available_languages
0,tt0047034,Godzilla,1954.0,Not Rated,96.0,"Takeo Murata, Ishirô Honda, Shigeru Kayama (st...",American nuclear weapons testing results in th...,['Japanese'],7.6,27485.0,...,0.0,2.0,5.0,7.0,NaN,0.0,0.0,0.0,0.0,1
1,tt0304141,Harry Potter and the Prisoner of Azkaban,2004.0,PG,142.0,"J.K. Rowling (novel), Steve Kloves (screenplay)","Harry Potter, Ron and Hermione return to Hogwa...",['English'],7.9,524511.0,...,0.0,3.0,6.0,4.0,1.0,0.0,2.0,0.0,0.0,1
2,tt0369436,Four Christmases,2008.0,PG-13,88.0,"Matt Allen (screenplay), Caleb Wilson (screenp...",A couple struggles to visit all four of their ...,['English'],5.7,61335.0,...,0.0,4.0,11.0,26.0,2.0,0.0,0.0,0.0,0.0,1
3,tt1077258,Planet Terror,2007.0,Not Rated,105.0,Robert Rodriguez,"After an experimental bio-weapon is released, ...",['English'],7.1,196149.0,...,0.0,3.0,6.0,13.0,NaN,0.0,0.0,0.0,0.0,1
4,tt0052182,Separate Tables,1958.0,Approved,100.0,"Terence Rattigan (screenplay), John Gay (scree...",The stories of several people are told as they...,['English'],7.4,6337.0,...,0.0,1.0,2.0,11.0,NaN,2.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,tt0266860,Pokemon 3: The Movie,2001.0,NaN,87.0,Satoshi Tajiri,"In the town of Greenfield, a young, lonely gir...",['Japanese'],5.9,12672.0,...,0.0,2.0,6.0,21.0,NaN,0.0,0.0,0.0,0.0,1
9303,tt0313487,Pokemon 4Ever,2002.0,NaN,75.0,Hideki Sonoda,Ash must stop a hunter who forces the mythical...,['Japanese'],5.7,7600.0,...,0.0,2.0,7.0,7.0,NaN,0.0,0.0,0.0,0.0,1
9304,tt0383974,Le chagrin et la pitie,1969.0,NaN,251.0,Marcel Ophuls,An in-depth exploration of the various reactio...,['French'],8.2,3484.0,...,0.0,2.0,9.0,18.0,NaN,0.0,0.0,0.0,0.0,1
9305,tt0134948,Just the Ticket,1999.0,NaN,115.0,Richard Wenk,A ticket scalper wants to make one last big de...,['English'],5.7,1822.0,...,0.0,1.0,2.0,26.0,NaN,0.0,0.0,0.0,0.0,1


In [51]:
omdb.columns

Index(['0', 'Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Oscars_won', 'Oscars_nominated', 'Golden_globe_won',
       'Golden_globe_nominated', 'Available_languages'],
      dtype='object')

In [52]:
ratings

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0
...,...,...,...,...
787536,812812,1032,tt0083530,3.0
787537,812813,99,tt0107798,3.0
787538,812814,333,tt0093857,3.0
787539,812815,49,tt0144168,3.0


In [53]:
 # Fill NaN Runtime
omdb.loc[:, 'Runtime'] = omdb.loc[:, 'Runtime'].fillna(omdb.loc[:, 'Runtime'].median())
# Fill NaN for imdbVotes
omdb.loc[:, 'imdbVotes'] = omdb.loc[:, 'imdbVotes'].fillna(omdb.loc[:, 'imdbVotes'].median())

In [54]:
#Combine awards in one column
omdb['Awards'] = omdb['Oscars_won'] + omdb['Golden_globe_won'] + omdb['Oscars_nominated'] + omdb['Golden_globe_nominated']
omdb = omdb.drop(columns={'Oscars_won', 'Oscars_nominated', 'Golden_globe_won', 'Golden_globe_nominated'})

In [55]:
features = omdb[['Year', 'Runtime', 'imdbVotes', 'Series', 'Awards']]
features

,Year,Runtime,imdbVotes,Series,Awards
0,1954.0,96.0,27485.0,0.0,0.0
1,2004.0,142.0,524511.0,0.0,2.0
2,2008.0,88.0,61335.0,0.0,0.0
3,2007.0,105.0,196149.0,0.0,0.0
4,1958.0,100.0,6337.0,0.0,2.0
...,...,...,...,...,...
9302,2001.0,87.0,12672.0,0.0,0.0
9303,2002.0,75.0,7600.0,0.0,0.0
9304,1969.0,251.0,3484.0,0.0,0.0
9305,1999.0,115.0,1822.0,0.0,0.0


## KNN

In [56]:
# Standardize the features
scaler = preprocessing.StandardScaler()
features = scaler.fit_transform(features)

In [57]:
features

array([[-1.8480496 , -0.37457098, -0.15497182, -0.08122467, -0.39494007],
       [ 0.86352638,  1.523969  ,  4.02362335, -0.08122467,  1.63080103],
       [ 1.08045246, -0.70475185,  0.12961178, -0.08122467, -0.39494007],
       ...,
       [-1.0345768 ,  6.0226833 , -0.35675294, -0.08122467, -0.39494007],
       [ 0.59236879,  0.40960857, -0.3707257 , -0.08122467, -0.39494007],
       [ 0.32121119, -0.20948055, -0.27881823, -0.08122467, -0.39494007]])

In [58]:
# features of the first entry (imdbID 439)
features[0]

array([-1.8480496 , -0.37457098, -0.15497182, -0.08122467, -0.39494007])

In [59]:
features.shape[0]

9307

In [60]:
# create NearesNeighbor model with the minkowski distance
knn = NearestNeighbors(metric='minkowski', p=2, algorithm='brute', n_jobs=-1)

In [61]:
# fit to all the features
knn.fit(features)

NearestNeighbors(algorithm='brute', n_jobs=-1)

In [62]:
# create two array for distance and index to store
distances = np.empty((features.shape[0],features.shape[0]))
indices = np.empty((features.shape[0],features.shape[0]))

# iterate over all imdbIDs
for i in range(0, features.shape[0]):
    # get all neighbors and the distances for each imdbID i
    distances[i], indices[i] = knn.kneighbors([features[i]], n_neighbors=features.shape[0])
    
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [63]:
distances

array([[0.00000000e+00, 1.05280658e-01, 1.40959605e-01, ...,
        2.00991885e+01, 2.43107624e+01, 2.46476435e+01],
       [0.00000000e+00, 2.80031556e-01, 3.78871682e-01, ...,
        1.81726995e+01, 2.24666033e+01, 2.33779322e+01],
       [2.10734243e-08, 7.02244009e-02, 7.36459216e-02, ...,
        1.97727565e+01, 2.44309140e+01, 2.47879718e+01],
       ...,
       [0.00000000e+00, 5.40991272e-01, 1.30845107e+00, ...,
        1.92557967e+01, 1.93464267e+01, 2.06224318e+01],
       [0.00000000e+00, 5.42469722e-02, 5.45382154e-02, ...,
        2.01258199e+01, 2.35400081e+01, 2.38258738e+01],
       [0.00000000e+00, 4.30400409e-02, 5.45374826e-02, ...,
        2.00821004e+01, 2.40539566e+01, 2.43612470e+01]])

In [64]:
indices

array([[0.000e+00, 4.474e+03, 7.451e+03, ..., 9.100e+01, 6.070e+03,
        7.882e+03],
       [1.000e+00, 4.410e+02, 7.350e+02, ..., 4.296e+03, 6.070e+03,
        7.882e+03],
       [2.000e+00, 5.877e+03, 6.767e+03, ..., 4.296e+03, 6.070e+03,
        7.882e+03],
       ...,
       [9.304e+03, 8.270e+03, 6.022e+03, ..., 2.535e+03, 7.882e+03,
        9.100e+01],
       [9.305e+03, 6.011e+03, 7.383e+03, ..., 9.100e+01, 6.070e+03,
        7.882e+03],
       [9.306e+03, 7.518e+03, 2.591e+03, ..., 9.100e+01, 6.070e+03,
        7.882e+03]])

In [65]:
len(indices)

9307

In [66]:
distances_df = pd.DataFrame(distances)
distances_df

,0,1,2,3,4,5,6,7,8,9,...,9297,9298,9299,9300,9301,9302,9303,9304,9305,9306
0,0.000000e+00,0.105281,0.140960,0.162895,0.179558,0.182973,0.199937,0.202530,0.206585,0.207685,...,15.756767,16.670637,17.160259,17.260525,17.394834,18.869935,19.471520,20.099188,24.310762,24.647643
1,0.000000e+00,0.280032,0.378872,0.420665,0.730627,0.795250,0.984072,1.005497,1.062379,1.111668,...,14.480780,14.688086,15.280145,15.986076,16.028065,16.492904,16.612724,18.172700,22.466603,23.377932
2,2.107342e-08,0.070224,0.073646,0.082545,0.100644,0.108714,0.118012,0.130329,0.137627,0.144069,...,15.410391,16.768338,17.266026,17.428966,17.553233,18.405156,19.766486,19.772756,24.430914,24.787972
3,0.000000e+00,0.056955,0.111056,0.137710,0.158383,0.177487,0.214012,0.216075,0.222155,0.231901,...,14.720940,16.359821,16.889746,17.004090,17.129940,17.198544,18.649412,19.129456,23.743611,24.230557
4,0.000000e+00,0.162876,0.174238,0.198314,0.248746,0.272846,0.294070,0.299865,0.301215,0.337855,...,15.213718,16.642386,16.660655,17.140916,17.152875,18.883353,19.394781,19.608726,24.170179,24.566253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,0.000000e+00,0.034528,0.040901,0.041390,0.041919,0.046727,0.054301,0.057605,0.059988,0.061999,...,15.774253,16.781969,17.287980,17.421780,17.560429,18.817344,19.788649,20.139729,24.507537,24.819134
9303,0.000000e+00,0.043040,0.051692,0.054988,0.073427,0.074239,0.078016,0.081470,0.083438,0.095714,...,15.893163,17.121347,17.605550,17.775677,17.911905,18.935082,20.242135,20.285812,24.936723,25.249106
9304,0.000000e+00,0.540991,1.308451,1.552037,1.639993,1.769932,1.771926,1.818569,1.843537,1.846346,...,15.659422,15.664735,15.677165,15.768764,15.862239,16.365611,19.147326,19.255797,19.346427,20.622432
9305,0.000000e+00,0.054247,0.054538,0.054712,0.055530,0.061206,0.061711,0.068182,0.068253,0.068818,...,15.731369,16.024926,16.582937,16.627124,16.770879,18.629500,18.780493,20.125820,23.540008,23.825874


In [67]:
indices_df = pd.DataFrame(indices)
indices_df

,0,1,2,3,4,5,6,7,8,9,...,9297,9298,9299,9300,9301,9302,9303,9304,9305,9306
0,0.0,4474.0,7451.0,366.0,7090.0,2790.0,6321.0,479.0,8692.0,4169.0,...,486.0,9179.0,7512.0,4815.0,6551.0,2535.0,4296.0,91.0,6070.0,7882.0
1,1.0,441.0,735.0,452.0,1195.0,526.0,446.0,662.0,192.0,405.0,...,5637.0,4835.0,91.0,4815.0,9179.0,6551.0,7512.0,4296.0,6070.0,7882.0
2,2.0,5877.0,6767.0,4706.0,4337.0,5820.0,5965.0,5308.0,6291.0,1365.0,...,486.0,9179.0,4815.0,7512.0,6551.0,2535.0,91.0,4296.0,6070.0,7882.0
3,3.0,1337.0,237.0,224.0,2336.0,2711.0,3625.0,477.0,7.0,2070.0,...,4835.0,9179.0,4815.0,7512.0,6551.0,2535.0,91.0,4296.0,6070.0,7882.0
4,4.0,5560.0,5855.0,3441.0,4490.0,7207.0,8433.0,8978.0,8826.0,5022.0,...,839.0,4815.0,9179.0,6551.0,7512.0,2535.0,4296.0,91.0,6070.0,7882.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,9302.0,1878.0,523.0,4898.0,7203.0,6587.0,1305.0,4719.0,2482.0,5242.0,...,486.0,9179.0,4815.0,7512.0,6551.0,2535.0,4296.0,91.0,6070.0,7882.0
9303,9303.0,6731.0,5599.0,7731.0,7254.0,8689.0,7481.0,4230.0,3285.0,267.0,...,486.0,9179.0,4815.0,7512.0,6551.0,2535.0,91.0,4296.0,6070.0,7882.0
9304,9304.0,8270.0,6022.0,3255.0,5504.0,7992.0,7881.0,4149.0,2637.0,5173.0,...,5415.0,6155.0,9299.0,839.0,6279.0,486.0,6070.0,2535.0,7882.0,91.0
9305,9305.0,6011.0,7383.0,5891.0,2554.0,5893.0,1731.0,7770.0,6606.0,5860.0,...,486.0,9179.0,4815.0,7512.0,6551.0,4296.0,2535.0,91.0,6070.0,7882.0


In [68]:
omdb = omdb.reset_index()
omdb

,index,0,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,...,imdbID,Rotten Tomatoes,Metacritic,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Awards
0,0,tt0047034,Godzilla,1954.0,Not Rated,96.0,"Takeo Murata, Ishirô Honda, Shigeru Kayama (st...",American nuclear weapons testing results in th...,['Japanese'],7.6,...,tt0047034,9.3,7.8,0.0,2.0,5.0,7.0,NaN,1,0.0
1,1,tt0304141,Harry Potter and the Prisoner of Azkaban,2004.0,PG,142.0,"J.K. Rowling (novel), Steve Kloves (screenplay)","Harry Potter, Ron and Hermione return to Hogwa...",['English'],7.9,...,tt0304141,9.0,8.2,0.0,3.0,6.0,4.0,1.0,1,2.0
2,2,tt0369436,Four Christmases,2008.0,PG-13,88.0,"Matt Allen (screenplay), Caleb Wilson (screenp...",A couple struggles to visit all four of their ...,['English'],5.7,...,tt0369436,2.4,4.1,0.0,4.0,11.0,26.0,2.0,1,0.0
3,3,tt1077258,Planet Terror,2007.0,Not Rated,105.0,Robert Rodriguez,"After an experimental bio-weapon is released, ...",['English'],7.1,...,tt1077258,7.4,NaN,0.0,3.0,6.0,13.0,NaN,1,0.0
4,4,tt0052182,Separate Tables,1958.0,Approved,100.0,"Terence Rattigan (screenplay), John Gay (scree...",The stories of several people are told as they...,['English'],7.4,...,tt0052182,6.9,NaN,0.0,1.0,2.0,11.0,NaN,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9302,9302,tt0266860,Pokemon 3: The Movie,2001.0,NaN,87.0,Satoshi Tajiri,"In the town of Greenfield, a young, lonely gir...",['Japanese'],5.9,...,tt0266860,NaN,NaN,0.0,2.0,6.0,21.0,NaN,1,0.0
9303,9303,tt0313487,Pokemon 4Ever,2002.0,NaN,75.0,Hideki Sonoda,Ash must stop a hunter who forces the mythical...,['Japanese'],5.7,...,tt0313487,NaN,NaN,0.0,2.0,7.0,7.0,NaN,1,0.0
9304,9304,tt0383974,Le chagrin et la pitie,1969.0,NaN,251.0,Marcel Ophuls,An in-depth exploration of the various reactio...,['French'],8.2,...,tt0383974,NaN,NaN,0.0,2.0,9.0,18.0,NaN,1,0.0
9305,9305,tt0134948,Just the Ticket,1999.0,NaN,115.0,Richard Wenk,A ticket scalper wants to make one last big de...,['English'],5.7,...,tt0134948,NaN,NaN,0.0,1.0,2.0,26.0,NaN,1,0.0


In [69]:
omdb_index = omdb[['index', '0']]
omdb_index

,index,0
0,0,tt0047034
1,1,tt0304141
2,2,tt0369436
3,3,tt1077258
4,4,tt0052182
...,...,...
9302,9302,tt0266860
9303,9303,tt0313487
9304,9304,tt0383974
9305,9305,tt0134948


In [70]:
ratings = ratings.merge(omdb_index, left_on= 'imdbID', right_on='0', how='left').drop(['0'], axis=1)

In [77]:
ratings = ratings[ratings['imdbID']!='tt0720339']

In [78]:
#Alles in eine Funktion
def predict_rating(imdbID, userID):
    zeitu = time.perf_counter()
    #print('Startzeit: '+str(zeitu))
    
    #Bestimme Index in omdb von der ImdbID
    index = omdb.loc[(omdb['imdbID'] == imdbID)].index
    zeit = time.perf_counter()
    zeitdiff = time.perf_counter()-zeitu
    #print('2: '+str(zeitdiff))
    
    #Speichere Reihenfolge der Indizes der Nachbarn des Films und tausche erste Reihe mit Spaltenbezeichnungen
    movie_row = indices_df.loc[index]
    movie_row = movie_row.reset_index(drop=True)
    movie_row.columns = movie_row.loc[0]
    movie_row.drop(0, inplace=True)
    movie_row.loc[1,:] = range(0,len(indices_df))
    zeitdiff = time.perf_counter()-zeit
    zeit = time.perf_counter()
    #print('3: '+str(zeitdiff))
    
    #Speichere alle Filme, die der User bewertet hat und bestimme index davon
    rated_id = ratings.loc[ratings['user_id'] == userID]
    rated_df = rated_id['index']
    rated_df = pd.DataFrame(rated_df)
    #print(rated_df)
    zeitdiff = time.perf_counter()-zeit
    zeit = time.perf_counter()
    #print('4: '+str(zeitdiff))
    
    # Bestimme die Position der bewerteten Filme innerhalb der Nachbarschaft
    position = movie_row[list(rated_df['index'].astype(int))]
    position = position.transpose()
    position.columns = ['position']
    position ['Index'] = position.index
    position = position.reset_index(drop = True)
    zeitdiff = time.perf_counter()-zeit
    zeit = time.perf_counter()
    #print('5: '+str(zeitdiff))
    
    #Greife auf den Film in der Distanzmatrix zu
    movie_row_distance = distances_df.loc[index]
    zeitdiff = time.perf_counter()-zeit
    zeit = time.perf_counter()
    #print('6: '+str(zeitdiff))
    
    #Bestimme Distanz der Filme vom Film, die der User bewertet hat
    user_distances = movie_row_distance[list(position['position'].astype(int))].transpose()
    user_distances.columns = ['distance']
    user_distances =user_distances.reset_index(drop = True)
    zeitdiff = time.perf_counter()-zeit
    zeit = time.perf_counter()
    #print('7: '+str(zeitdiff))
    
    #Füge alle Infos zusammen und sortiere von nächstem zum weitest entfernten Film
    neighbors = pd.concat([position, user_distances], axis=1, join = "inner")
    neighbors = neighbors.sort_values("distance")
    zeitdiff = time.perf_counter()-zeit
    zeit = time.perf_counter()
    #print('8: '+str(zeitdiff))
    
    #Füge die Ratings des Users zu diesen Filmen noch hinzu
    rated_id['Index']= rated_df
    neighbors = neighbors.merge(rated_id[["rating","Index"]], on="Index", how = "outer")
    neighbors = neighbors.iloc[1:]
    zeitdiff = time.perf_counter()-zeit
    zeit = time.perf_counter()
    #print('9: '+str(zeitdiff))
    
    # Berechne die Prognose: Gewichtung der Bewertung der Nachbarn je nach Distanz
    pred = sum(neighbors['rating']*(1+(1-neighbors['distance']/neighbors['distance'].mean()))) / neighbors.shape[0]
    zeitdiff = time.perf_counter()-zeit
    zeit = time.perf_counter()
    #print('10: '+str(zeitdiff))
    print('Gesamt: '+str(zeit-zeitu))
    
    return pred

In [79]:
n=100
test_set = ratings.sample(n)
test_set   

,Unnamed: 0,user_id,imdbID,rating,index
687992,709537,846,tt0120082,3.0,75.0
204879,210615,1912,tt0386117,4.0,710.0
661125,681642,847,tt0399201,2.5,1164.0
280016,288030,277,tt0347149,4.0,104.0
371799,382626,969,tt0111161,5.0,91.0
...,...,...,...,...,...
492922,507706,2003,tt0116040,2.5,2559.0
603709,622221,174,tt0061578,4.0,3089.0
383982,395176,221,tt0381707,1.5,3275.0
754961,778962,726,tt0126344,4.0,4328.0


In [80]:
predictions = np.empty(n)
reals = np.empty(n)
i = 0

for index, row in test_set.iterrows():
        imdbID = row['imdbID']
        userID = row['user_id']
        real = row['rating']
        pred = predict_rating(imdbID, userID)
        predictions[i] = pred
        reals[i] = real
        i = i+1
rmse = mean_squared_error(reals, predictions, squared=False)
print('RMSE: '+str(rmse))

<ipython-input-78-799712cb4ab0>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rated_id['Index']= rated_df


Gesamt: 0.1181203999999525
Gesamt: 0.06090119999998933
Gesamt: 0.03569720000018606
Gesamt: 0.032803600000079314
Gesamt: 0.029079800000090472
Gesamt: 0.028872299999875395
Gesamt: 0.042563800000152696
Gesamt: 0.0515831000000162
Gesamt: 0.0314963999999236
Gesamt: 0.028951999999890177
Gesamt: 0.024309200000061537
Gesamt: 0.027127800000016578
Gesamt: 0.031536000000187414
Gesamt: 0.0361840000000484
Gesamt: 0.030040999999982887
Gesamt: 0.026523399999859976
Gesamt: 0.025925099999994927
Gesamt: 0.02694019999989905
Gesamt: 0.028759300000047006
Gesamt: 0.045129599999881975
Gesamt: 0.02747599999997874
Gesamt: 0.027672400000028574
Gesamt: 0.026007899999967776
Gesamt: 0.02537539999980254
Gesamt: 0.024966400000039357
Gesamt: 0.028068699999948876
Gesamt: 0.03162459999998646
Gesamt: 0.04568180000001121
Gesamt: 0.10277589999986958
Gesamt: 0.08846390000007887
Gesamt: 0.10088749999999891
Gesamt: 0.08364140000003317
Gesamt: 0.09660619999999653
Gesamt: 0.06329809999988356
Gesamt: 0.0415487000000212
Gesamt: 

In [43]:
ratings[ratings['index'].isna()]

,Unnamed: 0,user_id,imdbID,rating,index
4169,4280,226,tt0720339,3.0,NaN
27008,27761,1359,tt0720339,3.0,NaN
131207,134786,1254,tt0720339,3.0,NaN
478516,492788,264,tt0720339,2.0,NaN
724051,746903,312,tt0720339,4.0,NaN


In [44]:
ratings[ratings['index'].notna()]

,Unnamed: 0,user_id,imdbID,rating,index
0,0,1264,tt0047034,3.5,0.0
1,1,213,tt0304141,2.5,1.0
2,2,593,tt0369436,3.0,2.0
3,3,609,tt1077258,4.0,3.0
4,4,1590,tt0052182,4.0,4.0
...,...,...,...,...,...
787536,812812,1032,tt0083530,3.0,1929.0
787537,812813,99,tt0107798,3.0,1097.0
787538,812814,333,tt0093857,3.0,2215.0
787539,812815,49,tt0144168,3.0,665.0
